In [3]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.metrics import classification_report
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
import nltk
from scipy.special import softmax

In [4]:
col_name = ['target','id', 'date','flag', 'user', 'text']
dataset_encode = "ISO-8859-1"
df = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding = dataset_encode,names=col_name)

In [5]:
df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
df['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [7]:
df['target']= df['target'].replace(4, 1)
df['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [9]:
def clean_text(text):
    
    stop_words = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", 
    "any", "are", "aren't", "as", "at", "be", "because", "been", "before", "being", 
    "below", "between", "both", "but", "by", "can't", "cannot", "could", "couldn't", 
    "did", "didn't", "do", "does", "doesn't", "doing", "don't", "down", "during", 
    "each", "few", "for", "from", "further", "had", "hadn't", "has", "hasn't", 
    "have", "haven't", "having", "he", "he'd", "he'll", "he's", "her", "here", 
    "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", 
    "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "isn't", "it", "it's", 
    "its", "itself", "let's", "me", "more", "most", "mustn't", "my", "myself", "no", 
    "nor", "not", "of", "off", "on", "once", "only", "or", "other", "ought", "our", 
    "ours", "ourselves", "out", "over", "own", "same", "shan't", "she", "she'd", 
    "she'll", "she's", "should", "shouldn't", "so", "some", "such", "than", "that", 
    "that's", "the", "their", "theirs", "them", "themselves", "then", "there", 
    "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", 
    "those", "through", "to", "too", "under", "until", "up", "very", "was", "wasn't", 
    "we", "we'd", "we'll", "we're", "we've", "were", "weren't", "what", "what's", 
    "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
    "why", "why's", "with", "won't", "would", "wouldn't", "you", "you'd", "you'll", 
    "you're", "you've", "your", "yours", "yourself", "yourselves"]
    
    def get_word_pos(tag):
        if tag.startswith('V'):
            return 'v'  # Verb
        elif tag.startswith('N'):
            return 'n'  # Noun
        elif tag.startswith('R'):
            return 'r'  # Adverb
        else:
            return 'n'  # Default to noun



    
    #convert to lower
    text = text.lower()
    
    #remove URL
    text = re.sub(r'(http[s]?://\S+|www\.\S+)', ' ', text)
    
    #replace @username with username 
    text = re.sub(r'@[\S]+', 'User', text)
    
    #remove hashtag sign
    text = re.sub(r'#(\S)+', r'\1', text)
    
    #remvoe numbers
    text = re.sub(r'\d+', '', text )
    
    #remove whitespaces
    text = re.sub(r'\s+', ' ', text)
    
    #strip
    text = text.strip()
    
    #remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    
    #tokenize
    tokenizer = RegexpTokenizer(r'\w+|[^\w\s]')
    tokens = tokenizer.tokenize(text)
    
    #pos
    pos_tag = nltk.pos_tag(tokens)
    
    #lemmatize 
    lemmatizer = WordNetLemmatizer()
    lammatized_token = [lemmatizer.lemmatize(token, get_word_pos(tag)) for token, tag in pos_tag]
    
    return ' '.join(lammatized_token)


In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [17]:
pos_samples = df[df['target']==1].sample(500, random_state=42)
neg_samples = df[df['target']==0].sample(500, random_state=42)

In [18]:
pos_samples.shape, neg_samples.shape

((500, 6), (500, 6))

In [19]:
sub_data = pd.concat([pos_samples, neg_samples])

In [20]:
sub_data.shape

(1000, 6)

In [22]:
sub_data['text'] = sub_data['text'].apply(clean_text)


In [33]:

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")


model.to(device)

config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [34]:
def predict_sentiment(text):
    encoded_input = tokenizer(text, return_tensors='pt').to(device)
    with torch.no_grad():
        output = model(**encoded_input)
    scores = output[0][0].cpu().numpy()
    scores = softmax(scores)

    negative_score = scores[0]
    positive_score = scores[2]
    return 1 if positive_score > negative_score else 0

In [35]:
sub_data['sentiment'] = sub_data['text'].apply(predict_sentiment)

In [36]:
sub_data.head()

,target,id,date,flag,user,text,sentiment
1012188,1,1881179620,Fri May 22 03:51:54 PDT 2009,NO_QUERY,tarawade,lookin ward long weekend really dont want go w...,0
1099036,1,1970537555,Sat May 30 04:15:49 PDT 2009,NO_QUERY,Millie_stillie,s music live meet people make,1
1275978,1,2001154935,Tue Jun 02 00:00:21 PDT 2009,NO_QUERY,zsangel,figure internet new ipod,1
1388988,1,2053074174,Sat Jun 06 03:10:08 PDT 2009,NO_QUERY,krisignacio,User wait worship guy tonight . it ' ll much fun,1
938859,1,1793548492,Thu May 14 03:06:55 PDT 2009,NO_QUERY,_DrInE_,User congrats james ! ! sure book go huge success,1


In [37]:
y_true = sub_data['target']
y_pred = sub_data['sentiment']
report = classification_report(y_true, y_pred, target_names=["negative", "positive"])
print(report)

              precision    recall  f1-score   support

    negative       0.77      0.61      0.68       500
    positive       0.68      0.82      0.74       500

    accuracy                           0.71      1000
   macro avg       0.72      0.71      0.71      1000
weighted avg       0.72      0.71      0.71      1000



In [38]:
def test_single_tweet(tweet):
    tweet = clean_text(tweet)
    sentiment = predict_sentiment(tweet)

   
    return "positive" if sentiment == 1 else "negative"

In [39]:
test_tweet = "I #hatedata science brain #dsbrain"
predicted_sentiment = test_single_tweet(test_tweet)
print(f"The sentiment of the tweet '{test_tweet}' is {predicted_sentiment}.")

The sentiment of the tweet 'I #hatedata science brain #dsbrain' is positive.
